In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python_speech_features
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator
import math
import numpy as np
from collections import defaultdict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5870 sha256=c41b859e6d6a99c05d3bca8ff7d9f54c7ce90a7844d2716037c752325c773272
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
Successfully built python_speech_features


In [6]:
def loadDataset(filename):
    dataset = []
    with open("/content/drive/MyDrive/Colab Notebooks/ProjetIA/my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
loadDataset("my.dat")

In [ ]:
def distance(instance1 , instance2 , k ):
    if not isinstance(instance1, tuple) or len(instance1) != 2:
        raise ValueError(f"instance1 doit être un tuple contenant exactement deux éléments. instance1={instance1}")
    
    if not isinstance(instance2, tuple) or len(instance2) != 2:
        raise ValueError(f"instance2 doit être un tuple contenant exactement deux éléments. instance2={instance2}")
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

In [ ]:

def getNeighbors(trainingSet , instance , k):
    distances =[]
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist)) 
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

 

In [ ]:
def nearestClass(neighbors):
    classVote ={}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1 
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [18]:
# Charger le dataset
dataset = loadDataset("/content/drive/MyDrive/Colab Notebooks/ProjetIA/my.dat")

# Définir les classes
results = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]

# Charger l'instance à classer
(rate, sig) = wav.read("/content/drive/MyDrive/Colab Notebooks/ProjetIA/321899__not_danny2462__huge-rocket-motor-startup.wav")
mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix, covariance, 0)

# Trouver les voisins les plus proches
neighbors = getNeighbors(dataset, feature, 5)

# Trouver la classe la plus proche
if neighbors:
    pred = nearestClass(neighbors)
else:
    pred = 0

print(results[pred])

<ipython-input-18-b0e46b7723a5>:8: WavFileWarning: Chunk (non-data) not understood, skipping it.
  (rate, sig) = wav.read("/content/drive/MyDrive/Colab Notebooks/ProjetIA/321899__not_danny2462__huge-rocket-motor-startup.wav")


blues
